In [1]:
import requests
import yaml
import time
from tqdm import tqdm
from datetime import datetime
import steamreviews
from time import sleep
import os
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [2]:
def review_insert(reviews, appid):
    need_data = ['votes_up', 'votes_funny', 'steam_purchase', 'received_for_free', 'written_during_early_access']
    need_data_dic ={'steam_appid': appid}
        
    for _, review in reviews[0]['reviews'].items():
        author = review['author']
        steamid = author['steamid']
        sql = "SELECT id FROM review WHERE steam_appid=%s AND steamid=%s" % (appid, steamid)
        if db_function.execute_select(sql):
            return
        
        for k, v in author.items():
            
            if k == 'last_played':
                v = datetime.fromtimestamp(v).strftime("%Y-%m-%d %H:%M:%S")
            need_data_dic[k] = v
        
        for col in need_data:
            need_data_dic[col] = review[col]
        
        placeholders = ', '.join(['%s'] * len(need_data_dic))
        columns = ', '.join(need_data_dic.keys())
        
        sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('review', columns, placeholders)
        db_function.execute_insert(sql, list(need_data_dic.values()))


In [3]:
start_time = time.time()
try_cnt = 0
db_function = None

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)

try:
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])

    sql = "SELECT steam_appid FROM game_info"
    app_ids = db_function.execute_select(sql)
    for appid in tqdm(app_ids):
        appid = appid[0]
        try_cnt +=1
        current_time = time.time()
        elapsed_time = current_time - start_time

        if try_cnt > 200 and elapsed_time < 300:
            sleep(60)
            try_cnt = 0
            start_time = time.time()
            

        
        request_params = dict()
        request_params['language'] = 'korean'

        reviews = steamreviews.download_reviews_for_app_id(appid,chosen_request_params=request_params)
        if reviews[0]['reviews']:
            review_insert(reviews, appid)
        sleep(5)
        
except:
    raise
    
finally:
    db_function.curs.close()
    db_function.conn.close()
    

  0%|                                                                                                                                                                                                                                                        | 0/32 [00:00<?, ?it/s]

[appID = 1829300] expected #reviews = 1


 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████               | 30/32 [02:37<00:10,  5.24s/it]

[appID = 1822190] expected #reviews = 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:47<00:00,  5.25s/it]
